In [19]:
from datasets import load_dataset, Dataset
import evaluate

In [38]:
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm
import os
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_fscore_support
num_folds = 5

In [3]:
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import pandas as pd
import numpy as np

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
dataset = load_dataset('json', data_files='lyrics.json')
dataset

Found cached dataset json (C:/Users/hsu/.cache/huggingface/datasets/json/default-ccc42051288650a4/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 82.15it/s]


DatasetDict({
    train: Dataset({
        features: ['valence', 'arousal', 'arousal_weight', 'lyrics', 'valence_weight'],
        num_rows: 11088
    })
})

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [82]:
#model_name = "google/pegasus-cnn_dailymail"
#model_name = "roberta-base"
#model_name = "facebook/bart-large-cnn"
#model_name = 't5-base'
#model_name = "distilbert-base-uncased"

In [6]:
from transformers import AutoTokenizer,BertForSequenceClassification, BertTokenizer, DistilBertModel, AutoModelForSequenceClassification
#tokenizer = RobertaTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

In [8]:
def convert_dataset(dataset, label = 'arousal'):
    input_encodings = tokenizer([str(x) for x in dataset["lyrics"]],max_length=512, 
                                truncation=True)
    #input_encodings = tokenizer([str(x) for x in dataset["lyrics"]])
    '''with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(dataset[label],max_length=128,
                                     truncation=True)
'''
    target_encodings = dataset[label]
    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings,
            "a_weight":dataset['arpis']
            
            }

Arousal

In [8]:
dataset_pt = dataset.map(convert_dataset,
                                       batched=True)

columns = ["input_ids", "labels", "attention_mask","a_weight"]
dataset_pt.set_format(type="torch", columns=columns)

Loading cached processed dataset at C:\Users\hsu\.cache\huggingface\datasets\json\default-ccc42051288650a4\0.0.0\e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4\cache-243a3d1327decf4a.arrow


In [9]:
dataset_pt = dataset_pt['train'].train_test_split(test_size=0.2)
dataset_pt

DatasetDict({
    train: Dataset({
        features: ['valence', 'arousal', 'arousal_weight', 'lyrics', 'valence_weight', 'input_ids', 'attention_mask', 'labels', 'a_weight'],
        num_rows: 8870
    })
    test: Dataset({
        features: ['valence', 'arousal', 'arousal_weight', 'lyrics', 'valence_weight', 'input_ids', 'attention_mask', 'labels', 'a_weight'],
        num_rows: 2218
    })
})

In [10]:
sampler = WeightedRandomSampler(dataset_pt['train']['a_weight'], len(dataset_pt['train']['a_weight']))

In [17]:
train_dataloader = DataLoader(dataset_pt['train'], batch_size=1, shuffle=False)
val_dataloader = DataLoader(dataset_pt['test'], batch_size=1, shuffle=False)

In [18]:
#model = RobertaModel.from_pretrained(model_name).to(device)
#model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity").to(device)
#model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

In [ ]:
class Transformer(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.transformer = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=16)
        

    def forward(self, x):
        x = self.transformer(x)
        
        
        return torch.abs(x)

In [19]:
from accelerate import Accelerator

accelerator = Accelerator()
model  = accelerator.prepare(model)

In [22]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

num_epochs = 10  # Set the number of training epochs

for epoch in tqdm(range(num_epochs)):
    #model.train()
    total_loss = 0
    train_loss = 0
    #gradient_update_count = 64
    batch_id = 0
    correct = 0
    for batch in train_dataloader:
        model.train()
        #with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        #decoder_input_ids = batch['decoder_input_ids'].to(device)
        #decoder_attention_mask = batch['decoder_attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        train_loss += loss.item()
        #print(outputs.logits)
        _,predict_label = torch.max(outputs.logits,1)
        correct += (predict_label==labels).sum()
        
        print('batch:', batch_id, 'loss:', loss.item(), end='\r')
        batch_id += 1
        '''if batch_id % gradient_update_count == 1:
            model.train()
            optimizer.zero_grad()
            print(train_loss / gradient_update_count)
            train_loss = torch.tensor(train_loss / gradient_update_count)
            train_loss = Variable(train_loss, requires_grad=True)
            train_loss.backward28
            optimizer.step()
            train_loss = 0
    '''
    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch: {epoch + 1}, Loss: {average_loss:.4f}", 'accuracy:', correct.item()/len(train_dataloader))
    #model.eval()

    '''batch_id = 0
    with torch.no_grad():
        for batch in val_dataloader:
            print('batch:', batch_id, end = '\r')
            rouge, bertscore = compute_metrics(batch, model)
            print(rouge[0], rouge[1], rouge[2], bertscore)
            batch_id += 1'''
    if not os.path.isdir('save_weights'):
            os.mkdir('save_weights')
    model.save_pretrained('save_weights/model'+str(epoch+1))
    tokenizer.save_pretrained('save_weights/tokenizer'+str(epoch+1))

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1, Loss: 0.5337 accuracy: 0.7501691093573845


 10%|█         | 1/10 [10:31<1:34:47, 631.93s/it]

Epoch: 2, Loss: 0.4814 accuracy: 0.7889515219842165


 20%|██        | 2/10 [21:09<1:24:42, 635.32s/it]

...

 80%|████████  | 8/10 [1:24:55<21:15, 637.74s/it]

Epoch: 9, Loss: 0.0838 accuracy: 0.9738444193912064


 90%|█████████ | 9/10 [1:35:34<10:37, 637.99s/it]

Epoch: 10, Loss: 0.0754 accuracy: 0.9774520856820744


100%|██████████| 10/10 [1:46:11<00:00, 637.12s/it]


In [33]:
model = BertForSequenceClassification.from_pretrained("save_weights/model4").to(device)
model.eval()
prediction = []
ans = []
#batch_id = 0
correct = 0
with torch.no_grad(): 
    for batch in tqdm(val_dataloader):
        input_ids =  batch['input_ids'].to(device)
        attention_mask =  batch['attention_mask'].to(device)
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        #outputs = model(**data)
        _,predict_label = torch.max(output.logits,1)
        correct += (predict_label==batch['labels'].to(device)).sum()
        #summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        #print(summary_text)
        prediction.append(predict_label.cpu().item())
        ans.append(batch['labels'].cpu().item())
        #batch_id += 1
        #print('batch:', batch_id)
print('accuracy:',correct.item() / len(val_dataloader))
print('f1 score:', f1_score(ans, prediction, average='macro'))
print('confusion matrix:\n', confusion_matrix(prediction,ans)) #[[tp,fp],[fn,tn]]

100%|██████████| 2218/2218 [00:35<00:00, 63.10it/s]

accuracy: 0.7628494138863842
f1 score: 0.6265376494434564
confusion matrix:
 [[ 176  142]
 [ 384 1516]]


Valence

In [9]:
def convert_dataset(dataset, label = 'valence'):
    input_encodings = tokenizer([str(x) for x in dataset["lyrics"]],max_length=512, 
                                truncation=True)
    #input_encodings = tokenizer([str(x) for x in dataset["lyrics"]])
    '''with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(dataset[label],max_length=128,
                                     truncation=True)
'''
    target_encodings = dataset[label]
    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings,
            "v_weight":dataset['valence_weight']
            
            }

In [10]:
dataset_pt = dataset.map(convert_dataset,
                                       batched=True)

columns = ["input_ids", "labels", "attention_mask","v_weight"]
dataset_pt.set_format(type="torch", columns=columns)

Map:   0%|          | 0/11088 [00:00<?, ? examples/s]

In [11]:
from sklearn.model_selection import KFold

In [12]:
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=3001)

In [13]:
dataset_pt

DatasetDict({
    train: Dataset({
        features: ['valence', 'arousal', 'arousal_weight', 'lyrics', 'valence_weight', 'input_ids', 'attention_mask', 'labels', 'v_weight'],
        num_rows: 11088
    })
})

In [170]:
dataset_pt = dataset_pt['train'].train_test_split(test_size=0.2)
dataset_pt

DatasetDict({
    train: Dataset({
        features: ['valence', 'arousal', 'arousal_weight', 'lyrics', 'valence_weight', 'input_ids', 'attention_mask', 'labels', 'v_weight'],
        num_rows: 8870
    })
    test: Dataset({
        features: ['valence', 'arousal', 'arousal_weight', 'lyrics', 'valence_weight', 'input_ids', 'attention_mask', 'labels', 'v_weight'],
        num_rows: 2218
    })
})

In [171]:
sampler = WeightedRandomSampler(dataset_pt['train']['v_weight'], len(dataset_pt['train']['v_weight']))

In [172]:
train_dataloader = DataLoader(dataset_pt['train'], batch_size=1, shuffle=False, sampler=sampler)
val_dataloader = DataLoader(dataset_pt['test'], batch_size=1, shuffle=False)

In [153]:
#model = RobertaModel.from_pretrained(model_name).to(device)
#model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity").to(device)
#model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

NameError: name 'model' is not defined

In [23]:
from accelerate import Accelerator


In [ ]:

accelerator = Accelerator()
model  = accelerator.prepare(model)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

num_epochs = 10  # Set the number of training epochs

for epoch in tqdm(range(num_epochs)):
    #model.train()
    total_loss = 0
    train_loss = 0
    #gradient_update_count = 64
    batch_id = 0
    correct = 0
    for batch in train_dataloader:
        model.train()
        #with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        #decoder_input_ids = batch['decoder_input_ids'].to(device)
        #decoder_attention_mask = batch['decoder_attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        train_loss += loss.item()
        #print(outputs.logits)
        _,predict_label = torch.max(outputs.logits,1)
        correct += (predict_label==labels).sum()
        
        print('batch:', batch_id, 'loss:', loss.item(), end='\r')
        batch_id += 1
        '''if batch_id % gradient_update_count == 1:
            model.train()
            optimizer.zero_grad()
            print(train_loss / gradient_update_count)
            train_loss = torch.tensor(train_loss / gradient_update_count)
            train_loss = Variable(train_loss, requires_grad=True)
            train_loss.backward28
            optimizer.step()
            train_loss = 0
    '''
    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch: {epoch + 1}, Loss: {average_loss:.4f}", 'accuracy:', correct.item()/len(train_dataloader))
    #model.eval()

    '''batch_id = 0
    with torch.no_grad():
        for batch in val_dataloader:
            print('batch:', batch_id, end = '\r')
            rouge, bertscore = compute_metrics(batch, model)
            print(rouge[0], rouge[1], rouge[2], bertscore)
            batch_id += 1'''
    if not os.path.isdir('save_weights'):
            os.mkdir('save_weights')
    model.save_pretrained('save_weights/model'+str(epoch+1))
    tokenizer.save_pretrained('save_weights/tokenizer'+str(epoch+1))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:04<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
train_set

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'v_weight'],
        num_rows: 5108
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'v_weight'],
        num_rows: 568
    })
})

In [74]:
def training(train_dataloader, val_dataloader,fold, num_epochs = 10, validation = True, val_monitor = 'loss'):
    model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity").to(device)
    accelerator = Accelerator()
    model  = accelerator.prepare(model)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    #num_epochs = 10  # Set the number of training epochs

    best_val_loss = np.inf
    best_val_accuracy = 0
    for epoch in tqdm(range(num_epochs)):
        #model.train()
        total_loss = 0
        train_loss = 0
        
        batch_id = 0
        correct = 0
        print(f"Epoch: {epoch + 1}",'training')
        for batch in train_dataloader:
            
            model.train()
            #with torch.no_grad():
            input_ids = torch.tensor([batch['input_ids']]).to(device)
            attention_mask = torch.tensor([batch['attention_mask']]).to(device)
            
            labels = batch['labels'].clone().detach().to(device)
            #print(labels)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            train_loss += loss.item()
            #print(outputs.logits)
            _,predict_label = torch.max(outputs.logits,1)
            correct += (predict_label==labels).sum()
            
            print('batch:', batch_id, 'loss:', loss.item(), end='\r')
            batch_id += 1
            
        average_loss = total_loss / len(train_dataloader)
        print(f"Loss: {average_loss:.4f}", 'accuracy:', correct.item()/len(train_dataloader))
        #model.eval()
        if validation:
            print('validation')
            model.eval()
            prediction = []
            ans = []
            val_loss = 0.0
            batch_id = 0
            correct = 0
            loss_function = torch.nn.CrossEntropyLoss()
            with torch.no_grad(): 
                for batch in val_dataloader:
                    input_ids =  torch.tensor([batch['input_ids']]).to(device)
                    attention_mask =  torch.tensor([batch['attention_mask']]).to(device)
                    labels = batch['labels'].clone().detach().to(device)
                    output = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                    )
                    #outputs = model(**data)
                    #loss = output.loss
                    loss = loss_function(output.logits, labels.to(device))
                    #if loss is None:
                    #    val_loss = np.inf
                    #else:
                    val_loss += loss.item()
                    _,predict_label = torch.max(output.logits,1)
                    correct += (predict_label==labels).sum()
                    
                    prediction.append(predict_label.cpu().item())
                    ans.append(labels.cpu().item())
                    print('batch:', batch_id, 'loss:', loss.item(), end='\r')
                    batch_id += 1
            val_loss /= len(val_dataloader)
            val_accuracy = correct.item() / len(val_dataloader)
            print('loss:', val_loss)
            print('accuracy:',val_accuracy)
            print('f1 score:', f1_score(ans, prediction, average='macro'))
            if val_monitor == 'loss':
                if val_loss <= best_val_loss:
                    best_val_loss = val_loss
                    if not os.path.isdir('save_weights'):
                        os.mkdir('save_weights')
                    if not os.path.isdir('save_weights/fold_{}'.format(fold)):
                        os.mkdir('save_weights/fold_{}'.format(fold))
                    model.save_pretrained('save_weights/fold_{}/model_best'.format(fold))
                    tokenizer.save_pretrained('save_weights/fold_{}/tokenizer_best'.format(fold))
            else:
                if val_accuracy >= best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    if not os.path.isdir('save_weights'):
                        os.mkdir('save_weights')
                    if not os.path.isdir('save_weights/fold_{}'.format(fold)):
                        os.mkdir('save_weights/fold_{}'.format(fold))
                    model.save_pretrained('save_weights/fold_{}/model_best'.format(fold))
                    tokenizer.save_pretrained('save_weights/fold_{}/tokenizer_best'.format(fold))
        else:
            if not os.path.isdir('save_weights'):
                    os.mkdir('save_weights')
            if not os.path.isdir('save_weights/fold_{}'.format(fold)):
                os.mkdir('save_weights/fold_{}'.format(fold))
            model.save_pretrained('save_weights/fold_{}/model'.format(fold) + str(epoch+1))
            tokenizer.save_pretrained('save_weights/fold_{}/tokenizer'.format(fold) + str(epoch+1))
    if not os.path.isdir('save_weights'):
        os.mkdir('save_weights')
    if not os.path.isdir('save_weights/fold_{}'.format(fold)):
        os.mkdir('save_weights/fold_{}'.format(fold))
    model.save_pretrained('save_weights/fold_{}/model_final'.format(fold))
    tokenizer.save_pretrained('save_weights/fold_{}/tokenizer_final'.format(fold))

In [72]:
def testing(test_loader,fold, load_best = True):
    print('testing')
    if load_best:
        model = BertForSequenceClassification.from_pretrained("save_weights/fold_{}/model_best".format(fold)).to(device)
    else:
        model = BertForSequenceClassification.from_pretrained("save_weights/fold_{}/model5".format(fold)).to(device)
    model.eval()
    prediction = []
    ans = []
    batch_id = 0
    correct = 0
    with torch.no_grad(): 
        for batch in test_dataloader:
            input_ids =  torch.tensor([batch['input_ids']]).to(device)
            attention_mask =  torch.tensor([batch['attention_mask']]).to(device)
            labels = torch.tensor(batch['labels'])
            output = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
            )
            #outputs = model(**data)
            _,predict_label = torch.max(output.logits,1)
            correct += (predict_label==labels.to(device)).sum()
            #summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            #print(summary_text)
            prediction.append(predict_label.cpu().item())
            ans.append(labels.cpu().item())
            batch_id += 1
            print('batch:', batch_id, end='\r')
    accuracy = correct.item() / len(test_dataloader)
    f1 = f1_score(ans, prediction, average='macro')
    conf_m = confusion_matrix(prediction,ans)
    return accuracy, f1, conf_m

In [73]:
test_accuracy = []
conf_m = []
#test_order_a = []
#pred_order_a = []
confusion_matrixs = []
test_macro_f1 = []

for i, (train, test) in tqdm(enumerate(kfold.split(dataset_pt['train']))):
    print('Fold {}:'.format(i+1))
    train_set = dataset_pt['train'][train]
    train_set = Dataset.from_dict(train_set)
    train_set = train_set.train_test_split(test_size=0.1)
    
    test_set = dataset_pt['train'][test]
    test_set = Dataset.from_dict(test_set)
    
    sampler = WeightedRandomSampler(train_set['train']['v_weight'], len(train_set['train']['v_weight']))
    train_dataloader = DataLoader(train_set['train'], batch_size=1, shuffle=False, sampler = sampler)
    val_dataloader = DataLoader(train_set['test'], batch_size=1, shuffle=False)
    test_dataloader = DataLoader(test_set, batch_size=1, shuffle=False)
    
    training(train_dataloader, val_dataloader,i+1, val_monitor = 'accuracy')
    
    accuracy, f1, confusion_m = testing(test_dataloader,i+1)
    test_accuracy.append(accuracy)
    test_macro_f1.append(f1)
    confusion_matrixs.append(confusion_m)

0it [00:00, ?it/s]

Fold 1:


0it [00:01, ?it/s]


OSError: textattack/bert-base-uncased is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [70]:
total_conf = confusion_matrixs[0]*0
for i in range(0,len(test_accuracy)):
    print("Fold {}:".format(i+1))
    print("accuracy:", test_accuracy[i])
    print("Confusion Matrix:")
    print(confusion_matrixs[i])
    total_conf += confusion_matrixs[i]
    print('macro F1 score :', test_macro_f1[i])
print("Total Confusion Matrix:\n",total_conf)
print("Avg accuracy:",np.array(test_accuracy).mean())
print("Avg macro f1 score:",np.array(test_macro_f1).mean())

Fold 1:
accuracy: 0.675834084761046
Confusion Matrix:
[[1299  387]
 [ 332  200]]
macro F1 score : 0.5703499426007813
Fold 2:
accuracy: 0.6803426510369702
Confusion Matrix:
[[1318  395]
 [ 314  191]]
macro F1 score : 0.5690896710276635
Fold 3:
accuracy: 0.6744815148782687
Confusion Matrix:
[[1307  432]
 [ 290  189]]
macro F1 score : 0.5636047525615872
Fold 4:
accuracy: 0.6860622462787551
Confusion Matrix:
[[1341  412]
 [ 284  180]]
macro F1 score : 0.5674350072662684
Fold 5:
accuracy: 0.6544880469102391
Confusion Matrix:
[[1270  398]
 [ 368  181]]
macro F1 score : 0.5446110231558353
Total Confusion Matrix:
 [[6535 2024]
 [1588  941]]
Avg accuracy: 0.6742417087730559
Avg macro f1 score: 0.5630180793224271


In [ ]:
model = BertForSequenceClassification.from_pretrained("save_weights/model5").to(device)
model.eval()
prediction = []
ans = []
#batch_id = 0
correct = 0
with torch.no_grad(): 
    for batch in tqdm(val_dataloader):
        input_ids =  batch['input_ids'].to(device)
        attention_mask =  batch['attention_mask'].to(device)
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        #outputs = model(**data)
        _,predict_label = torch.max(output.logits,1)
        correct += (predict_label==batch['labels'].to(device)).sum()
        #summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        #print(summary_text)
        prediction.append(predict_label.cpu().item())
        ans.append(batch['labels'].cpu().item())
        #batch_id += 1
        #print('batch:', batch_id)
print('accuracy:',correct.item() / len(val_dataloader))
print('f1 score:', f1_score(ans, prediction, average='macro'))
print('confusion matrix:\n', confusion_matrix(prediction,ans)) #[[tp,fp],[fn,tn]]

100%|██████████| 2218/2218 [00:34<00:00, 63.87it/s]


accuracy: 0.678990081154193
f1 score: 0.5587007064294017
confusion matrix:
 [[1332  386]
 [ 326  174]]
